# Which age group of animals has the highest adoption rate?

### 6 IV (age group), categorical

### 2 levels DV (adoptions), categorical

### Analysis Independent Chi-Square

# import packages

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels
import scipy
from scipy import stats
from scipy.stats import boxcox
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportions_chisquare

# read in data

In [8]:
shelter = pd.read_csv(r'C:\Users\hazre\OneDrive\OneNote\GitHub\Final-Project\MasterShelter.csv')

In [9]:
pd.set_option('display.max_columns', None)
shelter.head()

,speciesname,intakeMonth,intakeMonthR,id,movementMonth,DogAgeGroup,intakeYear,movementDate1R,intakereason,movementYear,movementtype,sexname,CatAgeGroup
0,Cat,11,February,15801,5,Senior,2009,May,Moving,2017,Adoption,Female,Senior
1,Dog,12,February,15932,4,Senior,2009,April,Moving,2017,Adoption,Male,Mature
2,Dog,8,February,28859,4,Mature,2012,April,Abandoned,2017,Adoption,Female,Mature
3,Dog,8,February,28859,2,Mature,2012,February,Abandoned,2020,Reclaimed,Female,Mature
4,Cat,1,January,30812,4,Senior,2013,April,Abandoned,2017,Foster,Female,Mature


# data wrangling

## create a dictionary to convert movementtype into 'adopted' and 'not adopted'

In [15]:
shelter.movementtype.value_counts()

0          12874
1          11236
Escaped        8
Name: movementtype, dtype: int64

In [23]:
dictionary = {'movementtype':
             {"Adoption" : 0,
             "Escaped" : 1,
             "Foster": 1,
             "Reclaimed": 1,
             "Released To Wild": 1,
             "Stolen": 1,
             "Transfer": 1,}}
shelter.replace(dictionary, inplace = True)

In [24]:
shelter.movementtype.value_counts()

0    12874
1    11244
Name: movementtype, dtype: int64

In [25]:
shelter.head()

,speciesname,intakeMonth,intakeMonthR,id,movementMonth,DogAgeGroup,intakeYear,movementDate1R,intakereason,movementYear,movementtype,sexname,CatAgeGroup
0,Cat,11,February,15801,5,Senior,2009,May,Moving,2017,0,Female,Senior
1,Dog,12,February,15932,4,Senior,2009,April,Moving,2017,0,Male,Mature
2,Dog,8,February,28859,4,Mature,2012,April,Abandoned,2017,0,Female,Mature
3,Dog,8,February,28859,2,Mature,2012,February,Abandoned,2020,1,Female,Mature
4,Cat,1,January,30812,4,Senior,2013,April,Abandoned,2017,1,Female,Mature


In [28]:
shelter.CatAgeGroup.value_counts()

Adult         11200
Adolescent     8234
Mature         1852
Kitten         1399
Senior         1050
Geriatric       290
Unknown          93
Name: CatAgeGroup, dtype: int64

### the unknown variable really should be removed- but I can't figure out how

### create a contingency table cats

In [26]:
shelter_crosstab = pd.crosstab(shelter['CatAgeGroup'], shelter['movementtype'])

In [27]:
shelter_crosstab

movementtype,0,1
CatAgeGroup,,
Adolescent,4345,3889
Adult,6623,4577
Geriatric,65,225
Kitten,532,867
Mature,952,900
Senior,356,694
Unknown,1,92


# run the independent chi square for cats

In [34]:
stats.chi2_contingency(shelter_crosstab)

(659.6650187514585,
 3.1171058934956828e-139,
 6,
 array([[4395.24487934, 3838.75512066],
        [5978.47251016, 5221.52748984],
        [ 154.79973464,  135.20026536],
        [ 746.77527158,  652.22472842],
        [ 988.58313293,  863.41686707],
        [ 560.48179783,  489.51820217],
        [  49.64267352,   43.35732648]]))

#### the pvalue is < .05 and there is a significant relationship between age groups and being adopted

# test assumptions of 5 cases per expected cell

#### all values are oer 5 so the assumption has been met

In [36]:
shelter.DogAgeGroup.value_counts()

Adult         10407
Adolescent     8234
Mature         1887
Puppy          1399
Senior         1060
Geriatric      1038
Unknown          93
Name: DogAgeGroup, dtype: int64

### again unknown should be removed

### create a contigency table dogs

In [37]:
shelter_crosstab = pd.crosstab(shelter['DogAgeGroup'], shelter['movementtype'])

In [38]:
shelter_crosstab

movementtype,0,1
DogAgeGroup,,
Adolescent,4345,3889
Adult,6184,4223
Geriatric,287,751
Mature,1026,861
Puppy,532,867
Senior,499,561
Unknown,1,92


# run the Independent Chi Square for dogs

In [39]:
stats.chi2_contingency(shelter_crosstab)

(682.450282115378,
 3.761851839148993e-144,
 6,
 array([[4395.24487934, 3838.75512066],
        [5555.17530475, 4851.82469525],
        [ 554.07629157,  483.92370843],
        [1007.26585952,  879.73414048],
        [ 746.77527158,  652.22472842],
        [ 565.81971971,  494.18028029],
        [  49.64267352,   43.35732648]]))

#### pvalue is <0.05 and there is a significant relationship between animal age and being adopted

# 